# Dataset Cleaning
To compare two pedestrian estimations with different methods - field count vs. cnn estimations, clean up and prepare a dataset from two sources.

In [1]:
import pandas as pd
import altair as alt
import numpy as np

alt.data_transformers.disable_max_rows()
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

# Get NYC field count 

metadata - http://www.nyc.gov/html/dot/downloads/pdf/bi-annual-ped-count-readme.pdf

It has three different sorts of count
* \_am: weekday morning (7-9am)
* \_pm: weekday afternoon (4-7pm)
* \_md: weekend midday (12-2pm)

In [32]:
query = ("https://data.cityofnewyork.us/resource/cqsj-cfgu.json")
df_wide = pd.read_json(query)
rename = dict()
for cname in df_wide.filter(regex='_am|_md|_pm').columns:
    rename[cname] = 'field-' + cname
df_wide = df_wide.rename(columns=rename)
df_wide.head()

,borough,the_geom,objectid,loc,street_nam,from_stree,to_street,index,field-may_07_am,field-may_07_pm,field-may_07_md,field-sept_07_am,field-sept_07_pm,field-sept_07_md,field-may_08_am,field-may_08_pm,field-may_08_md,field-sept_08_am,field-sept_08_pm,field-sept_08_md,field-may_09_am,field-may_09_pm,field-may_09_md,field-sept_09_am,field-sept_09_pm,field-sept_09_md,field-may_10_am,field-may_10_pm,field-may_10_md,field-sept_10_am,field-sept_10_pm,field-sept_10_md,field-may_11_am,field-may_11_pm,field-may_11_md,field-sept_11_am,field-sept_11_pm,field-sept_11_md,field-may_12_am,field-may_12_pm,field-may_12_md,field-sept_12_am,field-sept_12_pm,field-sept_12_md,field-may_13_am,field-may_13_pm,field-may_13_md,field-sept_13_am,field-sept_13_pm,field-sept_13_md,field-may_14_am,field-may_14_pm,field-may_14_md,field-sept_14_am,field-sept_14_pm,field-sept_14_md,field-may_15_am,field-may_15_pm,field-may_15_md,field-sept_15_am,field-sept_15_pm,field-sept_15_md,field-may_16_am,field-may_16_pm,field-may_16_md,field-sept_16_am,field-sept_16_pm,field-sept_16_md,field-may_17_am,field-may_17_pm,field-may_17_md,field-sept_17_am,field-sept_17_pm,field-sept_17_md,field-may_18_am,field-may_18_pm,field-may_18_md,field-sept_18_pm,field-sept_18_md,field-may_19_am,field-may_19_pm,field-may_19_md
0,Bronx,"{'type': 'Point', 'coordinates': [-73.90459140...",1,1,Broadway,West 231st Street,Naples Terrace,N,1189,4094,2508,734,2646,2939,802,4015,2631,1125,4310,3420,1001,3475,2832,991,4262,2469,1010,3609,3128,863,4119,2217,997,4440,2687,1328,3820,2428,1288,3328,3365,"1,268","4,315","2,276","1,210","4,710","3,825","1,206","4,590","3,008","1,220","4,384","2,641","1,450","4,646","2,996","1,788","4,980","3,033",1204,4520,2999,"1,246","4,531","2,686","1,309","3,642","2,830","1,916","5,893","2,776","1,111","4,044","2,731","1,271","4,502","2,899","4,464","2,967",-,-,-
1,Bronx,"{'type': 'Point', 'coordinates': [-73.92188432...",2,2,East 161st Street,Grand Concourse,Sheridan Avenue,Y,1511,3184,1971,1855,3754,1183,1136,2638,1522,1939,3283,1383,1351,3111,1304,1227,3137,2762,2077,3283,1409,1007,3069,1477,1734,3333,1772,2051,3525,1752,1233,1875,1912,"2,113","4,099","1,970","2,278","4,215","2,288","2,071","3,890","1,832","2,206","4,363","2,315","1,949","4,435","2,388","2,318","4,589","2,483",2005,4790,2512,"2,053","4,721","2,311","2,109","5,485","2,548","1,848","4,920","2,143","2,389","5,952","2,832","1,749","5,148","2,156","4,723","1,604","1,702","4,347","1,576"
2,Bronx,"{'type': 'Point', 'coordinates': [-73.89535781...",3,3,East Fordham Road,Valentine Avenue,Tiebout Avenue,Y,1832,12311,14391,1829,9215,15065,1061,8013,7927,2497,10010,8013,1689,8374,7472,2387,10655,6614,2285,10943,10424,2807,12658,7714,2289,15890,9167,2679,13548,8140,2125,7635,7022,"2,704","10,486","7,421","2,710","12,723","8,872","2,651","10,613","8,241","2,699","12,490","6,339","2,752","10,408","8,266","2,382","11,216","7,574",2733,10586,9072,"2,540","11,272","8,422","2,422","11,145","9,775","2,557","12,125","6,499","2,783","12,388","7,076","2,209","9,634","7,066","8,931","6,212","1,625","11,739","7,468"
3,Bronx,"{'type': 'Point', 'coordinates': [-73.87892467...",4,4,East Gun Hill Road,Bainbridge Avenue,Rochambeau Avenue,N,764,2673,1798,1360,1975,872,2102,3206,1989,3020,3915,1662,1536,2997,1672,2097,3135,1591,2954,3962,1717,1622,3006,1344,1661,2844,1510,1996,2836,1587,1906,2066,1261,"2,371","3,500","1,620","2,052","3,981","1,979","2,373","3,660","1,914","2,096","3,378","1,333","2,382","3,425","1,339","1,757","3,561","1,714",2256,2923,1574,"2,066","2,944","1,752","2,017","2,771","1,459","2,125","3,130","1,123","2,453","3,429","1,551","1,648","2,892","1,323","2,682","1,693",-,-,-
4,Bronx,"{'type': 'Point', 'coordinates': [-73.88956389...",5,5,East Tremont Avenue,Prospect Avenue,Clinton Avenue,N,650,2538,2334,1078,2739,2343,553,2438,1652,756,3095,1374,684,2660,2604,912,3637,2339,1075,5823,4085,1012,5785,3758,1093,6119,4274,1049,5859,3934,627,1503,2371,959,"3,140","2,273","1,007","4,175","2,854",760,"3,488","2,8

## Missing values

In [33]:
# Change missing value mark from '-' to NaN for easier handling
df_wide = df_wide.replace(to_replace = "-", value = np.nan)  

### missing value count - row-wise

In [34]:
# show me top 10
df_wide.isnull().sum(axis=1).sort_values(ascending=False).head(10)

101    19
28      9
113     4
103     4
86      4
88      4
112     4
100     4
102     4
98      4
dtype: int64

In [35]:
# 101 has 19 missing values. what is it?
df_wide.iloc[101]

borough                                            East River Bridges
the_geom            {'type': 'Point', 'coordinates': [-73.99094199...
objectid                                                          102
loc                                                               102
street_nam                              Manhattan Bridge - North Path
from_stree                                                   midpoint
to_street                                                         NaN
index                                                               N
field-may_07_am                                                   NaN
field-may_07_pm                                                   NaN
field-may_07_md                                                   NaN
field-sept_07_am                                                  NaN
field-sept_07_pm                                                  NaN
field-sept_07_md                                                  NaN
field-may_08_am     

In [36]:
# it has missing values in early years but recent years are ok. let's leave it as is.
# df_wide = df_wide.drop(df_wide.index[101])

### missing value count - column-wise

In [37]:
df_wide.isnull().sum(axis=0).sort_values(ascending=False).head(15)

field-may_19_md     64
field-may_19_am     64
field-may_19_pm     64
to_street           18
field-sept_08_md     2
field-sept_08_pm     2
field-may_07_am      2
field-may_07_pm      2
field-may_07_md      2
field-sept_08_am     2
field-may_08_pm      1
field-sept_09_md     1
field-sept_09_pm     1
field-sept_09_am     1
field-may_08_md      1
dtype: int64

In [38]:
df_wide.filter(regex='_(19|07|08|09)_(am|pm|md)').columns

Index(['field-may_07_am', 'field-may_07_pm', 'field-may_07_md',
       'field-sept_07_am', 'field-sept_07_pm', 'field-sept_07_md',
       'field-may_08_am', 'field-may_08_pm', 'field-may_08_md',
       'field-sept_08_am', 'field-sept_08_pm', 'field-sept_08_md',
       'field-may_09_am', 'field-may_09_pm', 'field-may_09_md',
       'field-sept_09_am', 'field-sept_09_pm', 'field-sept_09_md',
       'field-may_19_am', 'field-may_19_pm', 'field-may_19_md'],
      dtype='object')

In [39]:
# 2019 data (may_19_*) has too many missing values. Let's remove them.
# also remove 09, 08 and 07 since they have missing values and too old.
df_wide = df_wide.drop(columns=df_wide.filter(regex='_(19|07|08|09)_(am|pm|md)').columns)

## Convert to long format

In [44]:
df_long = pd.wide_to_long(df_wide, 
                          stubnames='field',
                          i='loc', 
                          j='time_measured', 
                          suffix='\w+', 
                          sep='-').reset_index()
df_long.head()

,loc,time_measured,from_stree,borough,objectid,the_geom,to_street,street_nam,index,field
0,1,may_10_am,West 231st Street,Bronx,1,"{'type': 'Point', 'coordinates': [-73.90459140...",Naples Terrace,Broadway,N,1010
1,2,may_10_am,Grand Concourse,Bronx,2,"{'type': 'Point', 'coordinates': [-73.92188432...",Sheridan Avenue,East 161st Street,Y,2077
2,3,may_10_am,Valentine Avenue,Bronx,3,"{'type': 'Point', 'coordinates': [-73.89535781...",Tiebout Avenue,East Fordham Road,Y,2285
3,4,may_10_am,Bainbridge Avenue,Bronx,4,"{'type': 'Point', 'coordinates': [-73.87892467...",Rochambeau Avenue,East Gun Hill Road,N,2954
4,5,may_10_am,Prospect Avenue,Bronx,5,"{'type': 'Point', 'coordinates': [-73.88956389...",Clinton Avenue,East Tremont Avenue,N,1075


## Split time_measured 
to year-month (datetime), and time (category: md/am/pm)

In [45]:
df_long['time_measured'] = df_long['time_measured'].replace(to_replace=r'^may_', value='05_', regex=True)
df_long['time_measured'] = df_long['time_measured'].replace(to_replace=r'^sept_', value='09_', regex=True)

split = df_long['time_measured'].str.rsplit(pat='_', n=1, expand = True)

df_long['measure_month'] = split[0]
df_long['measure_hour'] = split[1]
df_long['measure_month'] = pd.to_datetime(df_long['measure_month'], format="%m_%y")
df_long = df_long[['loc', 'measure_month', 'measure_hour', 'field']]
df_long['field'] = pd.to_numeric(df_long['field'].astype(str).str.replace(',',''), errors="coerce")
df_long.head()

,loc,measure_month,measure_hour,field
0,1,2010-05-01,am,1010
1,2,2010-05-01,am,2077
2,3,2010-05-01,am,2285
3,4,2010-05-01,am,2954
4,5,2010-05-01,am,1075


In [46]:
df_long.to_csv("../data/dataframe/nyc-field-count.csv", index=False)

In [47]:
len(df_long['loc'].unique())

114

## describing field count data

In [49]:
alt.Chart(df_long).mark_bar().encode(
    alt.X("field", 
          bin=alt.Bin(maxbins=100), 
          title='Field Count', 
          axis=alt.Axis(labelAngle=-45),
          scale=alt.Scale(domain=(0, 40000))),
    alt.Y('count()', title='Observations')
).properties(
    width=600, 
    height=150
).facet(
    row='measure_hour:N'
)

alt.FacetChart(...)

In [50]:
alt.Chart(df_long).mark_line().encode(
    alt.X("measure_month"),
    alt.Y('mean(field)'),
    color='measure_hour'
).properties(
    width=600, 
    height=150
)

alt.Chart(...)

## Note.
- Note that the NYC datatset has no entry for Sep. 2018 AM counts.
- NOte that am data is measured for 3 hours.
- What happend in May 2012 PM?

## Aggregation
We have multiple data points for 113 locations (remember that we dropeed one location - loc 102) measured from 2010 to 2018, May and September, and AM/MD/PM. We need to reduced muiltiple data points into one value  per each location so that we can compare the value to CNN based estimation. 

We are going to use 1) 2018 May and September average and 2) 3-year average (May/September 2016, May/September 2017, May/September 2018) since the GSVs are mostly taken between 2018 and 2019. 

Also, we don't know which hour is the best fit among AM/MD/PM. We are going to use the three AM/MD/PM and average of the three as well. 

To sum up,

(2018, 3-year) * (AM, MD, PM, ALL) gives us eight aggregated values for each location.


### 2018 one year

In [51]:
# average for am/md/pm
df_1year = df_long[df_long['measure_month'].between('2018-01-01', '2018-12-31')]\
    .groupby(["loc", "measure_hour"]).mean().unstack(level=1)
df_1year.columns = ['_'.join(col) +'_1y' for col in df_1year.columns]
# average across am/md/pm
df_1year['field_all_1y'] = df_long[df_long['measure_month'].between('2018-01-01', '2018-12-31')]\
    .groupby(["loc"]).mean()
df_1year.head()

,field_am_1y,field_md_1y,field_pm_1y,field_all_1y
loc,,,,
1,1271.0,2933.0,4483.0,3220.6
2,1749.0,1880.0,4935.5,3076.0
3,2209.0,6639.0,9282.5,6810.4
4,1648.0,1508.0,2787.0,2047.6
5,1016.0,2513.0,3771.0,2716.8


### 2016-2018 3-year

In [52]:
# average for am/md/pm
df_3year = df_long[df_long['measure_month'].between('2016-01-01', '2018-12-31')]\
    .groupby(["loc", "measure_hour"]).mean().unstack(level=1)
df_3year.columns = ['_'.join(col) +'_3y' for col in df_3year.columns]
# average across am/md/pm
df_3year['filed_all_3y'] = df_long[df_long['measure_month'].between('2016-01-01', '2018-12-31')]\
    .groupby(["loc"]).mean()
df_3year.head()

,field_am_3y,field_md_3y,field_pm_3y,filed_all_3y
loc,,,,
1,1370.6,2814.833333,4512.666667,2989.294118
2,2029.6,2265.666667,5158.166667,3217.117647
3,2502.2,7508.333333,10915.833333,7238.588235
4,2061.8,1483.500000,2974.666667,2179.882353
5,1034.4,2504.000000,3588.833333,2454.647059


In [53]:
# join two dataframes
df_field = df_1year.join(df_3year)
df_field.head()

,field_am_1y,field_md_1y,field_pm_1y,field_all_1y,field_am_3y,field_md_3y,field_pm_3y,filed_all_3y
loc,,,,,,,,
1,1271.0,2933.0,4483.0,3220.6,1370.6,2814.833333,4512.666667,2989.294118
2,1749.0,1880.0,4935.5,3076.0,2029.6,2265.666667,5158.166667,3217.117647
3,2209.0,6639.0,9282.5,6810.4,2502.2,7508.333333,10915.833333,7238.588235
4,1648.0,1508.0,2787.0,2047.6,2061.8,1483.500000,2974.666667,2179.882353
5,1016.0,2513.0,3771.0,2716.8,1034.4,2504.000000,3588.833333,2454.647059


# Construct variables from CNN dataset
Each row in the cnn dataset generated from the previous notebook has the score of the detection of a person in a image. Each location has 18 images; each image has multiple (0 to many) rows for identified person in the image. The score's range is 0.2 to 1.0 since our cut-off probability was 0.2.

The next step is assign one number to each location so that we can compare the number estimated from CNN to the number from field count.

We may think of several different approaches to find one number representing pedestrian volume estimated from GSV images. 

1) count the number of detections not less than a certain threshold, say, 0.8.
  - if a location has total 3 detections above 0.8, then the location's pedestrian volume is 3.
  - pro. simple, intuitive.
  - con. we need to decide the threshold; information from less then the threshold goes away.
  
2) add up all scores from detections 
  - if a location has total 5 detections and there scores are 0.9, 0.85, 0.85, 0.5, 0.2, then the location's pedestrian volume is 3.3.
  - pro. we use more information.
  - con. don't know how much the information from low score is useful.
  

We are going to construct 2 variables representing pedestrian volume of a location measured by faster RCNN on GSV images. 

  - 1) cnn_count: number of detections of which score is no less than 0.8
  - 2) cnn_sum_02: total sum of detection scores over 0.2 


In [54]:
df_cnn_long = pd.read_csv("../data/dataframe/nyc-cnn-exp.csv")\
    .rename(columns={'lid':'loc'})
# remove loc 102
# df_cnn_long = df_cnn_long.drop(df_cnn_long[df_cnn_long['loc']==102].index)
df_cnn_long.head()

,index,image,score,loc
0,0,image_23_80.jpg,0.999439,23
1,0,image_23_80.jpg,0.999275,23
2,0,image_23_80.jpg,0.996823,23
3,0,image_23_80.jpg,0.991345,23
4,0,image_23_80.jpg,0.988284,23


In [56]:
df_cnn = pd.DataFrame({
    'cnn_count':df_cnn_long[df_cnn_long['score'] >= 0.8].groupby('loc').size(),
    'cnn_sum': df_cnn_long[df_cnn_long['score'] >= 0.2].groupby('loc').sum()['score'],
#     'cnn_sum_05': df_cnn_long[df_cnn_long['score'] >= 0.5].groupby('loc').sum()['score'],
})
df_cnn = df_cnn.fillna(0)  # fill 0 where loc without count
df_cnn.head()

,cnn_count,cnn_sum
loc,,
1,43.0,58.994938
2,5.0,11.921850
3,97.0,166.952693
4,8.0,17.379551
5,39.0,67.678092


In [57]:
len(df_cnn)

114

## Join Field count and CNN estimation

In [58]:
df = df_field.join(df_cnn)
df = df.join(df_wide[['loc', 'index', 'borough']].set_index('loc').rename(columns={'index':'mayor_index'}))
print(len(df))
df.head()

114


,field_am_1y,field_md_1y,field_pm_1y,field_all_1y,field_am_3y,field_md_3y,field_pm_3y,filed_all_3y,cnn_count,cnn_sum,mayor_index,borough
loc,,,,,,,,,,,,
1,1271.0,2933.0,4483.0,3220.6,1370.6,2814.833333,4512.666667,2989.294118,43.0,58.994938,N,Bronx
2,1749.0,1880.0,4935.5,3076.0,2029.6,2265.666667,5158.166667,3217.117647,5.0,11.921850,Y,Bronx
3,2209.0,6639.0,9282.5,6810.4,2502.2,7508.333333,10915.833333,7238.588235,97.0,166.952693,Y,Bronx
4,1648.0,1508.0,2787.0,2047.6,2061.8,1483.500000,2974.666667,2179.882353,8.0,17.379551,N,Bronx
5,1016.0,2513.0,3771.0,2716.8,1034.4,2504.000000,3588.833333,2454.647059,39.0,67.678092,N,Bronx


In [59]:
df.to_csv("../data/dataframe/df_count.csv")